In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tensorflow.python.framework import ops
from tqdm import tqdm
import pickle
from tensorflow.core.protobuf import config_pb2

tf.reset_default_graph()

In [2]:
def data(i):
    training_data=[]
    training_data_bw=[]
    for j in range((i*124),(124+(i*124))):
        image = cv2.imread('D:\satellitedata\shufflenoisydata\%d.jpg' %(j))
        new_array=cv2.resize(image,(512,512))
        norm_img = np.zeros((512,512))
        final_img = cv2.normalize(new_array,  norm_img, 0, 255, cv2.NORM_MINMAX)
        new_array = final_img
        new_array_bw = cv2.cvtColor(new_array, cv2.COLOR_BGR2GRAY)
        new_array_bw = cv2.resize(new_array_bw,(512,512))
        new_array_bw = np.reshape(new_array_bw, (512,512,1))
        training_data.append(new_array)
        training_data_bw.append(new_array_bw)
    return training_data, training_data_bw

In [3]:
def label(i):
    label = []
    for j in range((i*124),(124+(i*124))):
        image = cv2.imread('D:\\satellitedata\\shufflelabel\\%d.jpg' %(j))
        new_array=cv2.resize(image,(512,512))
        norm_img = np.zeros((512,512))
        final_img = cv2.normalize(new_array, norm_img, 0, 255, cv2.NORM_MINMAX)
        new_array = final_img
        label.append(new_array)
    return label

In [4]:
# def discriminator(img,reuse=None):
#     with tf.variable_scope('dis',reuse=reuse):
#         flat = tf.contrib.layers.flatten(img)
#         # print(np.shape(flat))
#         hidden_1 = tf.layers.dense(inputs=flat,units=128, activation=tf.nn.leaky_relu, name='hidden_1')
#         hidden_2 = tf.layers.dense(inputs=hidden_1,units=128, activation=tf.nn.leaky_relu, name='hidden_2')
#         hidden_3=tf.layers.dense(inputs=hidden_2,units=128,activation=tf.nn.leaky_relu)
#         hidden_4=tf.layers.dense(inputs=hidden_3,units=128,activation=tf.nn.leaky_relu)
#         logits= tf.layers.dense(inputs=hidden_4, units=1)
#         output=tf.sigmoid(logits)
#     return output,logits

In [5]:
def discriminator(img,reuse=None):
    with tf.variable_scope('dis',reuse=reuse):
        hidden_1 = tf.layers.conv2d(img,20, (3,3),strides=(2, 2), padding='valid', activation=tf.nn.relu, use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
        hidden_2 = tf.layers.conv2d(hidden_1, 30, (3,3),strides=(2, 2), padding='valid', activation=tf.nn.relu, use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
        hidden_3= tf.layers.conv2d(hidden_2, 40, (3,3),strides=(2, 2), padding='valid', activation=tf.nn.relu, use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
        hidden_4= tf.layers.conv2d(hidden_3, 30, (5,5),strides=(2, 2), padding='valid', activation=tf.nn.relu, use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
        flat = tf.contrib.layers.flatten(hidden_4)
        hidden_5=tf.layers.dense(inputs=flat,units=500,activation=tf.nn.leaky_relu)
        logits= tf.layers.dense(inputs=hidden_5, units=1)
        output=tf.sigmoid(logits)
    return output,logits

In [6]:
# X_noisy = tf.placeholder(tf.float32, (None, 1024, 1024, 3), name='inputs')
# X_noisy_bw = tf.placeholder(tf.float32, (None, 1024, 1024, 1), name='inputs')
# X_label = tf.placeholder(tf.float32, (None, 1024, 1024, 3), name='targets')
# filters = {1:20,2:20,3:60, 4:60, 5:60, 6:60}

In [7]:
X_noisy = tf.placeholder(tf.float32, (None, 512, 512, 3), name='inputs')
X_noisy_bw = tf.placeholder(tf.float32, (None, 512, 512, 1), name='inputs')
X_label = tf.placeholder(tf.float32, (None, 512, 512, 3), name='targets')
filters = {1:20,2:20,3:60, 4:60, 5:60, 6:60}

In [8]:
def AutoEncoder(X_noisy, X_noisy_bw, reuse=None):
    with tf.variable_scope('gen',reuse=reuse):
        ### Encoder
        conv1 = tf.layers.conv2d(X_noisy, filters[1], (3,3),strides=(2, 2), padding='valid', activation=tf.nn.relu, use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
        conv2 = tf.layers.conv2d(X_noisy_bw, filters[2], (3,3),strides=(2,2),padding='valid',activation=tf.nn.relu, use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
        conv = tf.concat([conv1, conv2], 3)
        conv_shortcut = conv
        convD1 = tf.layers.conv2d(conv2,filters[4],(3,3), strides=(1, 1), padding='valid',dilation_rate=(2, 2), activation=tf.nn.relu,use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
        convD2 = tf.layers.conv2d(convD1,filters[5],(5,5), strides=(1, 1), padding='valid',dilation_rate=(3, 3), activation=tf.nn.relu,use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())        
        conv_shortcut1 = tf.layers.conv2d(conv_shortcut, 60, (17,17),strides=(1,1), padding='valid', activation=tf.nn.relu)
        print(np.shape(convD2))
        print(np.shape(conv_shortcut1))
        print(np.shape(conv_shortcut))
        convx = tf.math.add(convD2, conv_shortcut1)
        ###Decoder
        upsample1 = tf.image.resize_nearest_neighbor(convx, (384,384))
        conv4 = tf.layers.conv2d(upsample1, 40, (7,7), padding='same',activation=tf.nn.relu)
        upsample2 = tf.image.resize_nearest_neighbor(conv4, (440,440))
        conv5 = tf.layers.conv2d(upsample2, 20, (5,5), padding='same',activation=tf.nn.relu)
        upsample3 = tf.image.resize_nearest_neighbor(conv5, (512,512))
        conv6 = tf.layers.conv2d(upsample3, 3, (3,3), padding='same',activation=tf.nn.relu)
        print(np.shape(conv6))
        return conv6

In [9]:
conv6 = AutoEncoder(X_noisy, X_noisy_bw)
print(np.shape(conv6))

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `layer.__call__` method instead.
(?, 239, 239, 60)
(?, 239, 239, 60)
(?, 255, 255, 40)
(?, 512, 512, 3)
(?, 512, 512, 3)


In [10]:
D_output_real,D_logits_real=discriminator(X_label)
D_output_fake,D_logits_fake=discriminator(conv6,reuse=True)

def loss_func(logits_in,labels_in):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_in,labels=labels_in))

D_real_loss=loss_func(D_logits_real,tf.ones_like(D_logits_real)*0.9) #Smoothing for generalization
D_fake_loss=loss_func(D_logits_fake,tf.zeros_like(D_logits_real))

D_loss=D_real_loss+D_fake_loss

G_loss= loss_func(D_logits_fake,tf.ones_like(D_logits_fake))

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [11]:
lr=0.0001

#Do this when multiple networks interact with each other
tvars=tf.trainable_variables()  #returns all variables created(the two variable scopes) and makes trainable true
d_vars=[var for var in tvars if 'dis' in var.name]
g_vars=[var for var in tvars if 'gen' in var.name]

D_trainer=tf.train.AdamOptimizer(lr).minimize(D_loss,var_list=d_vars)
print(D_trainer)
G_trainer=tf.train.AdamOptimizer(lr).minimize(G_loss,var_list=g_vars)
print(G_trainer)

batch_size=124
totalsize = 21080
epochs=10
init=tf.global_variables_initializer()

name: "Adam"
op: "NoOp"
input: "^Adam/update_dis/conv2d/kernel/ApplyAdam"
input: "^Adam/update_dis/conv2d/bias/ApplyAdam"
input: "^Adam/update_dis/conv2d_1/kernel/ApplyAdam"
input: "^Adam/update_dis/conv2d_1/bias/ApplyAdam"
input: "^Adam/update_dis/conv2d_2/kernel/ApplyAdam"
input: "^Adam/update_dis/conv2d_2/bias/ApplyAdam"
input: "^Adam/update_dis/conv2d_3/kernel/ApplyAdam"
input: "^Adam/update_dis/conv2d_3/bias/ApplyAdam"
input: "^Adam/update_dis/dense/kernel/ApplyAdam"
input: "^Adam/update_dis/dense/bias/ApplyAdam"
input: "^Adam/update_dis/dense_1/kernel/ApplyAdam"
input: "^Adam/update_dis/dense_1/bias/ApplyAdam"
input: "^Adam/Assign"
input: "^Adam/Assign_1"

name: "Adam_1"
op: "NoOp"
input: "^Adam_1/update_gen/conv2d/kernel/ApplyAdam"
input: "^Adam_1/update_gen/conv2d/bias/ApplyAdam"
input: "^Adam_1/update_gen/conv2d_1/kernel/ApplyAdam"
input: "^Adam_1/update_gen/conv2d_1/bias/ApplyAdam"
input: "^Adam_1/update_gen/conv2d_2/kernel/ApplyAdam"
input: "^Adam_1/update_gen/conv2d_2/bias/

In [12]:
samples=[] #generator examples
Gcost = []
Dcost = []
minibatch_Dloss=0
minibatch_Gloss=0
gpu_options = tf.GPUOptions(allow_growth=True)
# session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

with tf.Session() as sess:
    sess.run(init, options=config_pb2.RunOptions(
        report_tensor_allocations_upon_oom=True))
    print("Session created")
    for epoch in tqdm(range(epochs)):
        num_batches = 170
        for i in range(num_batches):
#             X = training_data[i*batch_size:batch_size+(i*batch_size)]
            X, X_bw = data(i)
#             X_bw = training_data_bw[i*batch_size:batch_size+(i*batch_size)]
            Y = label(i)
#             Y = label[i*batch_size:batch_size+(i*batch_size)]
            
            _, temp_d_loss = sess.run([D_trainer,D_loss], feed_dict={X_label:Y,X_noisy:X, X_noisy_bw:X_bw})
            _, temp_g_loss = sess.run([G_trainer,G_loss], feed_dict={X_noisy:X, X_noisy_bw:X_bw})

            minibatch_Dloss += temp_d_loss / num_batches
            minibatch_Gloss += temp_g_loss / num_batches
        
            
        print("on epoch{}".format(epoch))
        # Print the cost every epoch
        print ("Cost after epoch %i: %f" % (epoch, minibatch_Dloss))
        print ("Cost after epoch %i: %f" % (epoch, minibatch_Gloss))
        Gcost.append(minibatch_Gloss)
        Dcost.append(minibatch_Dloss)
        
        # sample_z=np.random.uniform(-1,1,size=(1,100))
        # gen_sample=sess.run(generator(z,reuse=True),feed_dict={z:sample_z})
        
        # samples.append(gen_sample)

# plt.imshow(samples[0].reshape(1024,1024))
# plt.imshow(samples[99].reshape(1024,1024))

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Session created


  0%|                                                                                           | 0/10 [00:20<?, ?it/s]


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[62,384,384,60] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gen/ResizeNearestNeighbor (defined at e:\dl\satellite\env\lib\site-packages\tensorflow_core\python\framework\ops.py:1748) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[62,384,384,60] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gen/ResizeNearestNeighbor (defined at e:\dl\satellite\env\lib\site-packages\tensorflow_core\python\framework\ops.py:1748) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[add/_15]]
Hint: If you want to see a list o

In [ ]:
plt.plot(Gcost)

In [ ]:
plt.plot(Dcost)